# Equivalent Circuit Parameter Identification

This notebook provides example usage for identifying stationary parameters for a two RC branch Thevenin model. The Thevenin model represents an electrochemical battery through an empirical circuit model capable of capturing the electrical response of the battery. This model can be extended with a thermal submodel, as well as additional parallel resistor-capacitor branches.

### Setting up the Environment

Before we begin, we need to ensure that we have all the necessary tools. We will install PyBOP from its development branch and upgrade some dependencies:

In [ ]:
%pip install --upgrade pip ipywidgets -q
%pip install pybop -q

/home/nicola/GitHub/PyBOP/.nox/notebooks-overwrite/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.
/home/nicola/GitHub/PyBOP/.nox/notebooks-overwrite/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.


### Importing Libraries

With the environment set up, we can now import PyBOP alongside other libraries we will need:

In [ ]:
import numpy as np
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

Let's fix the random seed in order to generate consistent output during development, although this does not need to be done in practice.

In [ ]:
np.random.seed(8)

## Importing Parameters

This can be completed by importing a JSON representation, such as the one in the PyBOP [examples](https://github.com/pybop-team/PyBOP/blob/develop/examples/parameters/initial_ecm_parameters.json). To import via JSON, either download the example file, or create your own and update the path below to reference the corresponding file.

In [ ]:
# with open("examples/parameters/initial_ecm_parameters.json") as file:
#     parameter_values = pybamm.ParameterValues(json.load(file))

Alternatively, define the initial parameter set with a dictionary. Ensure you have definitions for all R's, C's, and initial overpotentials for any additional RC elements.

In this example, we use the default parameter value for the "Open-circuit voltage [V] as provided by the original PyBaMM class. To update this, provide a function definition that matches this [function](https://github.com/pybamm-team/PyBaMM/blob/1943aa5ab2895b5378220595923dbae3d66b13c9/pybamm/input/parameters/ecm/example_set.py#L17). First, we define the model class with two RC elements.

In [ ]:
model = pybamm.equivalent_circuit.Thevenin(options={"number of rc elements": 2})
parameter_values = pybamm.ParameterValues("ECM_Example")
parameter_values.update(
    {
        "Cell capacity [A.h]": 5,
        "Nominal cell capacity [A.h]": 5,
        "Current function [A]": 5,
        "Initial SoC": 0.5,
        "Element-1 initial overpotential [V]": 0,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 3.0,
        "R0 [Ohm]": 1e-3,
        "R1 [Ohm]": 2e-4,
        "C1 [F]": 1e4,
    }
)
# Optional arguments - only needed for two RC pairs
parameter_values.update(
    {
        "R2 [Ohm]": 0.0003,
        "C2 [F]": 40000,
        "Element-2 initial overpotential [V]": 0,
    },
    check_already_exists=False,
)

Let's create some synthetic data to identify the parameters. This data is then corrupted with a small amount of Gaussian noise to represent some additional uncertainty in the measured values. We can then form the `pybop.Dataset` from this data.

In [ ]:
sigma = 0.001
t_eval = np.arange(0, 900, 3)
sol = pybamm.Simulation(model, parameter_values=parameter_values).solve(t_eval=t_eval)
corrupt_values = sol["Voltage [V]"](t_eval) + np.random.normal(0, sigma, len(t_eval))

# Form dataset
dataset = pybop.Dataset(
    {
        "Time [s]": t_eval,
        "Current function [A]": sol["Current [A]"](t_eval),
        "Voltage [V]": corrupt_values,
    }
)

In this example, we are going to try to fit all five parameters at once. This isn't recommend for real-life application as identifiablity is challenging to guarantee with this large a parameter space. To do this, we define the `pybop.Parameters` as,

In [ ]:
parameter_values.update(
    {
        "R0 [Ohm]": pybop.ParameterDistribution(
            pybop.Gaussian(2 - 4, 1e-4, truncated_at=[1e-4, 1e-2]),
        ),
        "R1 [Ohm]": pybop.ParameterDistribution(
            pybop.Gaussian(1e-4, 1e-4, truncated_at=[1e-5, 1e-2]),
        ),
        "R2 [Ohm]": pybop.ParameterDistribution(
            pybop.Gaussian(1e-4, 1e-4, truncated_at=[1e-5, 1e-2]),
        ),
        "C1 [F]": pybop.ParameterDistribution(
            pybop.Gaussian(1e4, 5e2, truncated_at=[2.5e3, 5e4]),
        ),
        "C2 [F]": pybop.ParameterDistribution(
            pybop.Gaussian(1e4, 5e2, truncated_at=[2.5e3, 5e4]),
        ),
    }
)

The `Problem` class provides us with a single class that holds all of the objects we need to evaluate our selected `SumSquaredError` cost function. 

In [ ]:
simulator = pybop.pybamm.Simulator(
    model, parameter_values=parameter_values, protocol=dataset
)
cost = pybop.SumSquaredError(dataset)
problem = pybop.Problem(simulator, cost)

The cost function can be interrogated manually via the `problem([params])` API. In this example, that would look like the following,

In [ ]:
problem([0.001, 0.001, 5000, 0.001, 5000])

0.017001991250088822

Next, we construct the optimisation class with our algorithm of choice and run it. In this case, we select the CMA-ES method as it provides global optimisation capability. For the sake of reducing the runtime of this example, we limit the maximum iterations to 100; however, feel free to update this value.

In [ ]:
options = pybop.PintsOptions(
    max_unchanged_iterations=50,
    max_iterations=250,
)
optim = pybop.XNES(problem, options=options)
result = optim.run()

## Plotting and Visualisation

PyBOP provides various plotting utilities to visualize the results of the optimisation.

In [ ]:
pybop.plot.problem(problem, problem_inputs=result.x, title="Optimised Comparison");

### Convergence and Parameter Trajectories

To assess the optimisation process, we can plot the convergence of the cost function and the trajectories of the parameters:

In [ ]:
result.plot_convergence()
result.plot_parameters();

### Conclusion

This notebook illustrates how to perform parameter estimation using CMA-ES in PyBOP, providing insights into the optimisation process through various visualisations.